## Study :

### First Part

In this part, I tried to split the schools into different cohort based on various aspects(as mentioned below), and did a comparative analysis of students performance, percieved culture and student body demographics across the cohorts.

* Study : Schools service categories - Not all NYC school serve from pre-kindergarten/Kindergarten till 12. There are 4 predominant service catergories.

        a. Early chilhood school  b. Elementary school  c. Middle school  d. High school

Since, student apply for SHSAT in their 8th and 9th grade, middle schools contribution towards the SHSAT success are expected to be high as compared to other categories. Elementary schools contribution also need to be analysed separately. Interestingly also, more than 50% of the NYC schools serve till Elementary level. Therefore, a significant proportion of the entire student population do migrate each year from Elementary school to Middle school/Middle-High school.

* Study : Understand Economic Need Index(ENI) - More than 50% of NYC schools has ENI greater than 70%. Social and Economic status of students(population) highly impact their performance in schools. Therefore, it is imperative to measure that impact.

* Study : ELA and Math proficiency - There is a distinct group of schools where they are significantly lacking in ELA when theirs Math performance stands extremely good. Mainly because significant % of enrolled student's first language is not English. Race/Ethnicity distinction are also prominent between the groups.

In the very end, I tried to put a Conversion Scheme and Priority Ranking model.

### Second Part

In this part, I have tried to see the growth of district 5 schools enrollment, registration and participation in SHSAT over the years. And, the result clearly shows growth for most of them has decreased over the years with few of them being expection.

Additionally, I have used one external data source to compare how ELA proficiency has evolved over the years for various groups(By Ethnicity, Gender, Economic condition, English language learners etc.)

Note : Since no good SHSAT dataset is publicly available, I have taken a basic assumption of - Proficiency in ELA and Math (as determined by the US Common-Core standards) at each grade(3-8) represent students growth and acts as an indicator for a students success in the SHSAT. However, this will be a serapate study to prove the hypothesis - 'Proficient students(in ELA and Math) are more successful in the SHSAT'.

###### Acknowledgement : I do appreciate all other kernel authors and their works here. I try to follow them mostly and they heavily motivates my study here.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import scale
%matplotlib inline

In [ ]:
### Read : School data 2016
school_data = pd.read_csv(r'../input/data-science-for-good/2016 School Explorer.csv')

### Reda : D5 SHSAT
d5_shsat_data = pd.read_csv(r'../input/data-science-for-good/D5 SHSAT Registrations and Testers.csv')

### Read : NYC school meals income levels [Source : Socrata]
nyc_school_meals_income_levels = pd.read_csv(r'../input/nyc-school-meals-income-levels/nyc-school-meals-income-levels.csv')

### Read : School ELA Results 2013-2017 (Public)

school_results = pd.read_csv(r'../input/school-ela-results-20132017-public/School ELA Results 2013-2017 (Public)_consolidated.csv')

### Used datasets
print('School data 2016 : ',school_data.shape, '\nDistrict 5 SHSAT data : ', d5_shsat_data.shape,'\nNYC school meals income levels : ', nyc_school_meals_income_levels.shape, '\nNYC School ELA Results(2013-2017) : ',school_results.shape)

In [ ]:
############### Pre-processing ###############

### Grade Low/Grade High - conversion to type string
school_data['Grade Low'] = school_data['Grade Low'].astype(str)
school_data['Grade High'] = school_data['Grade High'].astype(str)

### String conversion to decimal
def string2decimal(s):
    if isinstance(s, str):
        s = s.strip()
        s = s.replace(",", "")
        s = s.replace("$", "")
        s = s.replace("%", "")
        return float(s)
    else:
        return s

school_data['School Income Estimate'] = school_data['School Income Estimate'].apply(lambda x: string2decimal(x))
school_data['Percent ELL'] = school_data['Percent ELL'].apply(lambda x: string2decimal(x)/100)
school_data['Percent Asian'] = school_data['Percent Asian'].apply(lambda x: string2decimal(x)/100)
school_data['Percent Black'] = school_data['Percent Black'].apply(lambda x: string2decimal(x)/100)
school_data['Percent Hispanic'] = school_data['Percent Hispanic'].apply(lambda x: string2decimal(x)/100)
school_data['Percent Black / Hispanic'] = school_data['Percent Black / Hispanic'].apply(lambda x: string2decimal(x)/100)
school_data['Percent White'] = school_data['Percent White'].apply(lambda x: string2decimal(x)/100)
school_data['Student Attendance Rate'] = school_data['Student Attendance Rate'].apply(lambda x: string2decimal(x)/100)
school_data['Percent of Students Chronically Absent'] = school_data['Percent of Students Chronically Absent'].apply(lambda x: string2decimal(x)/100)
school_data['Rigorous Instruction %'] = school_data['Rigorous Instruction %'].apply(lambda x: string2decimal(x)/100)
school_data['Collaborative Teachers %'] = school_data['Collaborative Teachers %'].apply(lambda x: string2decimal(x)/100)
school_data['Supportive Environment %'] = school_data['Supportive Environment %'].apply(lambda x: string2decimal(x)/100)
school_data['Effective School Leadership %'] = school_data['Effective School Leadership %'].apply(lambda x: string2decimal(x)/100)
school_data['Strong Family-Community Ties %'] = school_data['Strong Family-Community Ties %'].apply(lambda x: string2decimal(x)/100)
school_data['Trust %'] = school_data['Trust %'].apply(lambda x: string2decimal(x)/100)




school_results['Mean Scale Score'] = school_results['Mean Scale Score'].apply(lambda x: int(x) if x != 's' else 's')
school_results['Level 1%'] = school_results['Level 1%'].apply(lambda x: float(x) if x != 's' else 0.1)
school_results['Level 2%'] = school_results['Level 2%'].apply(lambda x: float(x) if x != 's' else 0.1)
school_results['Level 3%'] = school_results['Level 3%'].apply(lambda x: float(x) if x != 's' else 0.1)
school_results['Level 4%'] = school_results['Level 4%'].apply(lambda x: float(x) if x != 's' else 0.1)
school_results['Level 3+4%'] = school_results['Level 3+4%'].apply(lambda x: float(x) if x != 's' else 0.1)

school_results[['Level 1%', 'Level 2%', 'Level 3%', 'Level 4%', 'Level 3+4%']] = school_results[['Level 1%', 'Level 2%', 'Level 3%', 'Level 4%', 'Level 3+4%']].replace(0, 0.1)


**Study: School service categories**

    INFO :
    Early childhood School - pk,0k,1,2,3
    Elementary School - pk,0k,1,2,3,4,5,6
    Middle Schools - 5,6,7,8
    High School - 9,10,11,12

1. (209+409+19+7)/1272 = 51% NYC schools serve Pre-kindergarten(PK)/Kindergarten(0K) till grade 5/6 [Early childhood plus Elementary School]
2. 22% NYC schools serve grade 5/6 till grade 8 [Middle School]
3. 14% NYC schools serve Pre-kindergarten(PK)/Kindergarten(0K) till grade 8 [Early childhood plus Elementary plus Middle School]
4. 7% NYC schools serve  grade 5/6 till grade 12 [Middle plus High School]
5. 2% NYC schools serve Pre-kindergarten(PK)/Kindergarten(0K) till grade 12 [Early childhood plus Elementary plus Middle plus High School]
6. 1.5% NYC schools serves as only Early childhood Schools.

Since students apply and appear for SHSAT in their 8th or 9th grade, all middle school's(All schools under point 2,3,4,5) environment, culture and curriculum(rigor) should significantly contribute to ones exam performance. Therefore, they need to be analysed as a separate group. However, impact of Elementary Schools should also be analysed separately (specifically schools under point 1) since they lay the founding stone of a students career path. We also realize, a major chunk of the entire student population migrate each year from Elementary school to Middle school/Middle-High school.

In [ ]:
plt.figure(figsize=(10,7))
service_grade_categories = school_data[['SED Code', 'Grade Low', 'Grade High']].pivot_table(values = 'SED Code'
                                        , index = 'Grade Low', columns = 'Grade High', aggfunc = np.size, fill_value = 0)
plt.title('Field value represents # of schools\n', fontsize=16)
sns.heatmap(service_grade_categories, annot = True, fmt = 'd', linewidths=.2, cmap="YlGnBu", )

In [ ]:
school_data['Early_Childhood'] = 0
school_data['Elementary'] = 0
school_data['Middle'] = 0
school_data['High'] = 0

school_data['Early_Childhood']  = np.where(school_data['Grade Low'].isin(['PK', '0K']) & school_data['Grade High'].isin(['0K', '02', '03', '04']), 1, \
                                           np.where(school_data['Grade Low'].isin(['PK', '0K']) & school_data['Grade High'].isin(['05', '06']), 1, \
                                                    np.where(school_data['Grade Low'].isin(['PK', '0K']) & school_data['Grade High'].isin(['08']), 1, \
                                                             np.where(school_data['Grade Low'].isin(['PK', '0K']) & school_data['Grade High'].isin(['12']), 1, 0))))

school_data['Elementary'] = np.where(school_data['Grade Low'].isin(['PK', '0K']) & school_data['Grade High'].isin(['05', '06']), 1, \
                                    np.where(school_data['Grade Low'].isin(['01', '02', '03', '04']) & school_data['Grade High'].isin(['05', '06']), 1, \
                                            np.where(school_data['Grade Low'].isin(['PK', '0K']) & school_data['Grade High'].isin(['08']), 1, \
                                                    np.where(school_data['Grade Low'].isin(['PK', '0K']) & school_data['Grade High'].isin(['12']), 1, 0))))

school_data['Middle'] = np.where(school_data['Grade Low'].isin(['PK', '0K']) & school_data['Grade High'].isin(['08']), 1, \
                                 np.where(school_data['Grade Low'].isin(['PK', '0K']) & school_data['Grade High'].isin(['12']), 1, \
                                          np.where(school_data['Grade Low'].isin(['04', '05', '06']) & school_data['Grade High'].isin(['08']), 1, \
                                                   np.where(school_data['Grade Low'].isin(['05', '06', '07']) & school_data['Grade High'].isin(['12']), 1, 0))))

school_data['High'] = np.where(school_data['Grade Low'].isin(['PK', '0K']) & school_data['Grade High'].isin(['12']), 1, \
                                np.where(school_data['Grade Low'].isin(['05', '06', '07']) & school_data['Grade High'].isin(['10', '12']), 1,\
                                         np.where(school_data['Grade Low'].isin(['09']) & school_data['Grade High'].isin(['12']), 1, 0)))

**Study : Understand Economic Need Index**

Economic Need Index(ENI) reflects the socio-economic position of the school as being represented by its student population. Given by the below formula -

Economic Need Index = (Percent Temporary Housing) + (Percent HRA-eligible X 0.5) + (Percent Free Lunch Eligible X 0.5)

Where, 

1. Students are identified in Temporary housing if they are identified in temporary housing in last 4 years. Highest weightage has been given to this factor while calculating ENI.
2. Students are identified as HRA-eligible if their families have been identified by the Human Resource Administration(HRA) as receiving certain types of public assistance.
3. Student are identified as free lunch eligible based on the below table. Then also, all public school students are free lunch elibigle.

Clearly, higher ENI means poor socio-economic condition of a school population.

In [ ]:
nyc_school_meals_income_levels.head()

1. Majority(> 50%) of the NYC schools socio-economic conditions are poor(> 73%) [ Ref : Density Plot : Economic Need Index ].
2. "School's Income Estimate" also strongly works as a proxy for socio-economic condition. Model for School Income Estimate are unclear. [ Ref : Correlation : Economic Need Index Vs. School Income Estimate ]

In [ ]:
print("Median ENI (%) : ", 100*school_data['Economic Need Index'].median())
print("Mean ENI (%) : ", 100*school_data['Economic Need Index'].mean())
print("Median School Income Estimate ($) : ", school_data['School Income Estimate'].median())
print("Mean School Income Estimate ($) : ", school_data['School Income Estimate'].mean())

In [ ]:
f,axes=plt.subplots(1,3,figsize=(22,5))

sns.kdeplot(school_data['Economic Need Index'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[0])
axes[0].set_title('Density Plot : Economic Need Index')
sns.kdeplot(school_data['School Income Estimate'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[1])
axes[1].set_title('Density Plot : School Income Estimate')
sns.regplot(x="Economic Need Index", y="School Income Estimate", data=school_data[['Economic Need Index', 'School Income Estimate']], ax = axes[2])
axes[2].set_title('Correlation : Economic Need Index Vs. School Income Estimate')

**Study : Impact of Economic Need Index on student's performance**

1. Social and Economic status of students(population) highly impact their exam performance. [ Ref : Correlation : Economic Need Index Vs. Average Math Proficiency/ELA Proficiency ]
2. However, there are few schools where students are able to produce 'Proficient' and 'Above Proficient' results (First quadrant) despite they have 
very high Economic Need Index (or very poor socio-economic condition). We would like to study them as a separate group and compare them with other cohort.[ Ref : Correlation : Economic Need Index Vs. Average Math Proficiency/ELA Proficiency ]

In [ ]:
f,axes=plt.subplots(1,2,figsize=(18,5),sharex=True)
sns.regplot(x="Economic Need Index", y="Average ELA Proficiency", data=school_data[['Economic Need Index', 'Average ELA Proficiency']], ax = axes[0])
axes[0].axvline(x = 0.7, color = 'r')
axes[0].axhline(y = 3.0, color = 'r')
axes[0].set_title('Correlation : Economic Need Index Vs. Average ELA Proficiency')
sns.regplot(x="Economic Need Index", y="Average Math Proficiency", data=school_data[['Economic Need Index', 'Average Math Proficiency']], ax = axes[1])
axes[1].axvline(x = 0.7, color = 'r')
axes[1].axhline(y = 3.0, color = 'r')
axes[1].set_title('Correlation : Economic Need Index Vs. Average Math Proficiency')

    INFO : Each year, New York State administers exams in English Language Arts (ELA) and mathematics to students across the state in grades 3 through 8. 
    In addition to fulfilling federal and state mandates, these Common Core-aligned exams also help gauge year-to-year progress for students and schools. 
    The Proficiency Rating, which can range from 1.0 – 4.5, shows where a student fell within a particular Performance Level. 
    For example, a Proficiency Rating of 2.5 would indicate that a student scored at Performance Level 2 with a scale score that is halfway between 
    the lowest Performance Level 2 score and the lowest Performance Level 3 score. For Performance Level 4, the highest possible Proficiency Rating is 4.5.
    
    NYS Level 4 : Above Proficient
    NYS Level 3 : Proficient
    NYS Level 2 : Below Proficient
    NYS Level 1 : Well below Proficient
                                                                                         [Source : Understanding of ELA, Individual Student Report - NYC DOE]
                                                                                       
Extected [Average ELA proficiency(for all the schools)] : 2.53

Extected [Average Math proficiency(for all the schools)] : 2.66

ENI = 0.65 intutively choosen(close to mean) from the ENI distribution as a hard cut off to define High ENI and Low ENI group. However, more granular analysis can also be done for individual ENI level.

In [ ]:
### Ignore the missing part in the distribution
f,axes=plt.subplots(1,2,figsize=(18,5),sharex=True)

sns.kdeplot(school_data['Average ELA Proficiency'].fillna(0), shade  = True, color = 'r', ax = axes[0])
axes[0].set_title('Density Plot : Average ELA proficiency')

sns.kdeplot(school_data['Average Math Proficiency'].fillna(0), shade  = True, color = 'b', ax = axes[1])
axes[1].set_title('Density Plot : Average Math proficiency')

In [ ]:
school_data_high_eni_ela_proficient = school_data[(school_data['Economic Need Index'] >= 0.65) & (school_data['Average ELA Proficiency'] >= 3)]
school_data_low_eni_ela_proficient = school_data[(school_data['Economic Need Index'] < 0.65) & (school_data['Average ELA Proficiency'] >= 3)]

school_data_high_eni_math_proficient = school_data[(school_data['Economic Need Index'] >= 0.65) & (school_data['Average Math Proficiency'] >= 3)]
school_data_low_eni_math_proficient = school_data[(school_data['Economic Need Index'] < 0.65) & (school_data['Average Math Proficiency'] >= 3)]

Comparision of Avg. ELA and Avg. Math proficiency shown below. Comparision drawn for proficiency level of 3 and 4 for High ENI vs. Low ENI schools(based on cutoff ENI = 0.65). Estimated ELA proficiency tends to be better in High ENI schools (poor socio-economic condition). Estimated Math proficiency seems to be almost same compared to Low ENI schools.

In [ ]:
f,axes=plt.subplots(1,2,figsize=(18,5),sharex=True)

sns.kdeplot(school_data_high_eni_ela_proficient['Average ELA Proficiency'].fillna(0), shade  = True, color = 'b', ax = axes[0])
sns.kdeplot(school_data_low_eni_ela_proficient['Average ELA Proficiency'].fillna(0), shade  = True, color = 'y', ax = axes[0])
axes[0].set_title('Density Plot Comparision : Average ELA Proficiency ≥ 3\nBlue : High ENI   Yellow : Low ENI')

sns.kdeplot(school_data_high_eni_math_proficient['Average Math Proficiency'].fillna(0), shade  = True, color = 'b', ax = axes[1])
sns.kdeplot(school_data_low_eni_math_proficient['Average Math Proficiency'].fillna(0), shade  = True, color = 'y', ax = axes[1])
axes[1].set_title('Density Plot Comparision : Average Math Proficiency ≥ 3\nBlue : High ENI   Yellow : Low ENI')

In [ ]:
school_data_high_eni_ela_below_proficient = school_data[(school_data['Economic Need Index'] >= 0.65) & (school_data['Average ELA Proficiency'] < 3)]
school_data_low_eni_ela_below_proficient = school_data[(school_data['Economic Need Index'] < 0.65) & (school_data['Average ELA Proficiency'] < 3)]

school_data_high_eni_math_below_proficient = school_data[(school_data['Economic Need Index'] >= 0.65) & (school_data['Average Math Proficiency'] < 3)]
school_data_low_eni_math_below_proficient = school_data[(school_data['Economic Need Index'] < 0.65) & (school_data['Average Math Proficiency'] < 3)]

Comparision of Avg. ELA and Avg. Math proficiency shown below.  Comparision drawn for proficiency level of 1 and 2 for High ENI vs. Low ENI schools(based on cutoff ENI = 0.65). 
Based on estimation, it seems (and it is intuitively obvious, as we already have realized this above in ENI vs. Avg ELA/Math proficiency) majority of the High ENI school's require significant improvement in ELA and Math.

In [ ]:
f,axes=plt.subplots(1,2,figsize=(18,5),sharex=True)

sns.kdeplot(school_data_high_eni_ela_below_proficient['Average ELA Proficiency'].fillna(0), shade  = True, color = 'b', ax = axes[0])
sns.kdeplot(school_data_low_eni_ela_below_proficient['Average ELA Proficiency'].fillna(0), shade  = True, color = 'y', ax = axes[0])
axes[0].set_title('Density Plot Comparision : Average ELA Proficiency ≤ 3\nBlue : High ENI   Yellow : Low ENI')

sns.kdeplot(school_data_high_eni_math_below_proficient['Average Math Proficiency'].fillna(0), shade  = True, color = 'b', ax = axes[1])
sns.kdeplot(school_data_low_eni_math_below_proficient['Average Math Proficiency'].fillna(0), shade  = True, color = 'y', ax = axes[1])
axes[1].set_title('Density Plot Comparision : Average Math Proficiency ≤ 3\nBlue : High ENI   Yellow : Low ENI')

Categorizing schools based on ELA and Math proficiency.

Proficiency :

1 = Proficient(>= 3) in both ELA and Math

2 = Proficient(>= 3) in ELA but not in Math

3 = Proficient(>= 3) in Math but not int ELA

4 = Proficient(>= 3) in none

In [ ]:
school_data['proficiency'] = np.where((school_data['Average ELA Proficiency'] >= 3) & (school_data['Average Math Proficiency'] >= 3), 1,
                                np.where((school_data['Average ELA Proficiency'] >= 3) & (school_data['Average Math Proficiency'] < 3), 2,
                                    np.where((school_data['Average ELA Proficiency'] < 3) & (school_data['Average Math Proficiency'] >= 3), 3, 4)))

    INFO :
    The City provides educational services to students who speak a language other than English at home and are not yet proficient in English. If a school does not have a bilingual program, parents may select another school in the same school district that does have a program.
                                                              [Source : English Language Learners(ELL) and Bilingual program, NYC.gov]

143 NYC schools are both proficient in ELA and Math. They have very low % of English Language Learners (ELL) (since most of the students are English speaking and by default proficient in English) and tends to have 'White' dominance. Within this group, there is a subgroup who has higher socio-economic need.

154 NYC schools are proficient in Math but not in ELA. This group of schools tends to have higher % of ELL (since most of the students are non-English speaking and learners).

3 NYC schools are proficient in ELA not not in Math.

972 NYC schools are proficient in none. This group is predominantly represented by Black/Black-Hispanic. All community schools falls under this category (That justifies the purpose of community schools as well).

In [ ]:
f,axes=plt.subplots(4,2,figsize=(24,22))

school_data['proficiency'].hist(ax = axes[0,0])
axes[0,0].set_title('Distribution of Proficiency Levels (Discrete)\n1: Proficient in both    2: Proficient only in ELA    3: Proficient only in Math    4: Proficient in None')

sns.kdeplot(100*school_data[school_data['proficiency'] == 1]['Percent ELL'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[0,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 3]['Percent ELL'].fillna(0), cut = 0, shade  = True, color = 'g', ax = axes[0,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 4]['Percent ELL'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[0,1])
axes[0,1].set_title('\nDensity Plot : Percent ELL\nBlue : Proficient in both    Green : Proficient only in Math   Red : Proficient in None')

sns.kdeplot(100*school_data[school_data['proficiency'] == 1]['Percent White'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[1,0])
sns.kdeplot(100*school_data[school_data['proficiency'] == 3]['Percent White'].fillna(0), cut = 0, shade  = True, color = 'g', ax = axes[1,0])
sns.kdeplot(100*school_data[school_data['proficiency'] == 4]['Percent White'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[1,0])
axes[1,0].set_title('\nDensity Plot : Percent White\nBlue : Proficient in both    Green : Proficient only in Math   Red : Proficient in None')

sns.kdeplot(100*school_data[school_data['proficiency'] == 1]['Percent Black'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[1,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 3]['Percent Black'].fillna(0), cut = 0, shade  = True, color = 'g', ax = axes[1,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 4]['Percent Black'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[1,1])
axes[1,1].set_title('\nDensity Plot : Percent Black\nBlue : Proficient in both    Green : Proficient only in Math   Red : Proficient in None')

sns.kdeplot(100*school_data[school_data['proficiency'] == 1]['Percent Asian'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[2,0])
sns.kdeplot(100*school_data[school_data['proficiency'] == 3]['Percent Asian'].fillna(0), cut = 0, shade  = True, color = 'g', ax = axes[2,0])
sns.kdeplot(100*school_data[school_data['proficiency'] == 4]['Percent Asian'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[2,0])
axes[2,0].set_title('\nDensity Plot : Percent Asian\nBlue : Proficient in both    Green : Proficient only in Math   Red : Proficient in None')

sns.kdeplot(100*school_data[school_data['proficiency'] == 1]['Percent Hispanic'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[2,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 3]['Percent Hispanic'].fillna(0), cut = 0, shade  = True, color = 'g', ax = axes[2,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 4]['Percent Hispanic'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[2,1])
axes[2,1].set_title('\nDensity Plot : Percent Hispanic\nBlue : Proficient in both    Green : Proficient only in Math   Red : Proficient in None')

sns.kdeplot(100*school_data[school_data['proficiency'] == 1]['Percent Black / Hispanic'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[3,0])
sns.kdeplot(100*school_data[school_data['proficiency'] == 3]['Percent Black / Hispanic'].fillna(0), cut = 0, shade  = True, color = 'g', ax = axes[3,0])
sns.kdeplot(100*school_data[school_data['proficiency'] == 4]['Percent Black / Hispanic'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[3,0])
axes[3,0].set_title('\nDensity Plot : Percent Black/Hispanic\nBlue : Proficient in both    Green : Proficient only in Math   Red : Proficient in None')

sns.kdeplot(100*school_data[school_data['proficiency'] == 1]['Economic Need Index'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[3,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 3]['Economic Need Index'].fillna(0), cut = 0, shade  = True, color = 'g', ax = axes[3,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 4]['Economic Need Index'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[3,1])
axes[3,1].set_title('\nDensity Plot : Percent Economic Need Index\nBlue : Proficient in both    Green : Proficient only in Math   Red : Proficient in None')


In [ ]:
school_data[school_data['Community School?'] == 'Yes']['proficiency'].value_counts()

**Study : Survey Analysis**

    INFO :

    The 6 elements of the framework for great schools -
    1. Rigorous Instruction
    2. Collaborative Teachers
    3. Supportive Environment
    4. Effective School Leadership
    5. Strong Family-Community Ties
    6. Trust

    Rigorous Instruction %  : Survery response (Overall % positive)

        Do teachers believe that the school implements a high quality curriculum—aligned to the Common Core—that challenges students, scaffolds their experience, and engages students?
        Do students believe that the school implements a high quality curriculum—aligned to the Common Core—that challenges students, scaffolds their experience, and is enlivened by teachers who are engaged?

    Collaborative Teachers % : Survery response (Overall % positive)

        Do parents feel that teachers are committed to student success, improving their schools, and continuous professional learning within a culture of respect?
        Do students feel that teachers are committed to student success, improving their schools, and continuous professional learning within a culture of respect?

    Supportive Environment % : Survery response (Overall % positive)

        Do teachers feel that the school creates a culture that encourages students to support one another and provides for their social and emotional growth?
        Do students feel that the school creates a culture that encourages students to support one another and provides for their social and emotional growth?

    Effective School Leadership % : Survery response (Overall % positive)

        How do parents feel about the school leadership’s ability to develop a climate of inclusiveness, inspire staff and students with a clear vision for student achievement, and seek input of all members of the school community?
        How do teachers feel about the school leadership’s ability to develop a climate of inclusiveness, inspire staff and students with a clear vision for student achievement, and seek input of all members of the school community?

    Strong Family-Community Ties % : Survery response (Overall % positive)

        How do parents feel about communication with school staff and the school’s efforts to engage parents as partners in their child’s education?
        How do teachers feel about communication with parents and the school’s efforts to engage parents as partners in their child’s education? 

    Trust % : Survery response (Overall % positive)

        Do parents report feelings of mutual respect and value within the school community?
        Do teachers report feelings of mutual respect and value within the school community?
                    [Source : NYC School Survey Report 2015-2016 ]

We understand these 6 elements creates preception(in 6 distinct dimensions) of a school's rigor, collaboration, culture and environment from the POV of teachers, students and parents.

In [ ]:
f,axes=plt.subplots(4,2,figsize=(24,20))

sns.kdeplot(100*school_data[school_data['proficiency'] == 1]['Rigorous Instruction %'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[0,0])
sns.kdeplot(100*school_data[school_data['proficiency'] == 3]['Rigorous Instruction %'].fillna(0), cut = 0, shade  = True, color = 'g', ax = axes[0,0])
sns.kdeplot(100*school_data[school_data['proficiency'] == 4]['Rigorous Instruction %'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[0,0])
axes[0,0].set_title('\nDensity Plot : Survey Response - Rigorous Instruction %\nBlue : Proficient in both    Green : Proficient only in Math   Red : Proficient in None')

sns.kdeplot(100*school_data[school_data['proficiency'] == 1]['Collaborative Teachers %'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[0,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 3]['Collaborative Teachers %'].fillna(0), cut = 0, shade  = True, color = 'g', ax = axes[0,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 4]['Collaborative Teachers %'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[0,1])
axes[0,1].set_title('\nDensity Plot : Survey Response - Collaborative Teachers %\nBlue : Proficient in both    Green : Proficient only in Math   Red : Proficient in None')

sns.kdeplot(100*school_data[school_data['proficiency'] == 1]['Supportive Environment %'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[1,0])
sns.kdeplot(100*school_data[school_data['proficiency'] == 3]['Supportive Environment %'].fillna(0), cut = 0, shade  = True, color = 'g', ax = axes[1,0])
sns.kdeplot(100*school_data[school_data['proficiency'] == 4]['Supportive Environment %'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[1,0])
axes[1,0].set_title('\nDensity Plot : Survey Response - Supportive Environment %\nBlue : Proficient in both    Green : Proficient only in Math   Red : Proficient in None')

sns.kdeplot(100*school_data[school_data['proficiency'] == 1]['Effective School Leadership %'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[1,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 3]['Effective School Leadership %'].fillna(0), cut = 0, shade  = True, color = 'g', ax = axes[1,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 4]['Effective School Leadership %'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[1,1])
axes[1,1].set_title('\nDensity Plot : Survey Response - Effective School Leadership %\nBlue : Proficient in both    Green : Proficient only in Math   Red : Proficient in None')

sns.kdeplot(100*school_data[school_data['proficiency'] == 1]['Strong Family-Community Ties %'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[2,0])
sns.kdeplot(100*school_data[school_data['proficiency'] == 3]['Strong Family-Community Ties %'].fillna(0), cut = 0, shade  = True, color = 'g', ax = axes[2,0])
sns.kdeplot(100*school_data[school_data['proficiency'] == 4]['Strong Family-Community Ties %'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[2,0])
axes[2,0].set_title('\nDensity Plot : Survey Response - Strong Family-Community Ties %\nBlue : Proficient in both    Green : Proficient only in Math   Red : Proficient in None')

sns.kdeplot(100*school_data[school_data['proficiency'] == 1]['Trust %'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[2,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 3]['Trust %'].fillna(0), cut = 0, shade  = True, color = 'g', ax = axes[2,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 4]['Trust %'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[2,1])
axes[2,1].set_title('\nDensity Plot : Survey Response - Trust %\nBlue : Proficient in both    Green : Proficient only in Math   Red : Proficient in None')

sns.kdeplot(100*school_data[school_data['proficiency'] == 1]['Economic Need Index'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[3,0])
sns.kdeplot(100*school_data[school_data['proficiency'] == 3]['Economic Need Index'].fillna(0), cut = 0, shade  = True, color = 'g', ax = axes[3,0])
sns.kdeplot(100*school_data[school_data['proficiency'] == 4]['Economic Need Index'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[3,0])
axes[3,0].set_title('\nDensity Plot : Economic Need Index\nBlue : Proficient in both    Green : Proficient only in Math   Red : Proficient in None')

sns.kdeplot(100*school_data[school_data['proficiency'] == 1]['School Income Estimate'].fillna(0), cut = 0, shade  = True, color = 'b', ax = axes[3,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 3]['School Income Estimate'].fillna(0), cut = 0, shade  = True, color = 'g', ax = axes[3,1])
sns.kdeplot(100*school_data[school_data['proficiency'] == 4]['School Income Estimate'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[3,1])
axes[3,1].set_title('\nDensity Plot : School Income Estimate\nBlue : Proficient in both    Green : Proficient only in Math   Red : Proficient in None')


## Conversion and Priority Ranking Scheme 

Now that we have got 3 distinct groups -

* Group 1 -  Fully proficient in ELA and Math.
* Group 2 - Proficient in Math but not in ELA.
* Group 3 - Proficient in none.

We can build a scheme to  - 

* Convert candidate of Group 3 to Group 2 [Atleast make the schools proficient in one(Math) where currently they are proficient in none]
* Convert candidate of Group 2 to Group 1 [Make the schools proficient in both where currently they are proficient in one(Math)]

Conversion and Priority Ranking Model [ Group 2 --> Group 1 ]

1. Given each candidate from Group 1, find out K-nearest neighbours from Group 2 based on the dimensions representing school service categories, Economic Need index, student body demographics, perceived environment and culture of the school. Compare each of the dimensions of the given candidate to its K neighbours to deduce conversion strategy.

2. Rank candidates in Group 2 based on number of times it was selected as a nearest neighbour. Higher the number of selection means it is close to many number of schools who are doing good.

In [ ]:
######################### Pre-processing #########################

##### Drop missing values #####
##### Dropped because missing in most of the columns #####
school_data_with_missing_values = pd.DataFrame()
school_data_with_missing_values = school_data_with_missing_values.append([school_data[~pd.notnull(school_data['Economic Need Index'])]])
school_data = school_data[pd.notnull(school_data['Economic Need Index'])]


##### Impute missing values #####
##### Strings with "missing", Numerical with mean #####
school_data['Student Achievement Rating'] = school_data['Student Achievement Rating'].fillna('missing')
school_data['Supportive Environment Rating'] = school_data['Supportive Environment Rating'].fillna('missing')
school_data['Collaborative Teachers Rating'] = school_data['Collaborative Teachers Rating'].fillna('missing')
school_data['Rigorous Instruction Rating'] = school_data['Rigorous Instruction Rating'].fillna('missing')
school_data['Trust Rating'] = school_data['Trust Rating'].fillna('missing')
school_data['Strong Family-Community Ties Rating'] = school_data['Strong Family-Community Ties Rating'].fillna('missing')
school_data['Effective School Leadership Rating'] = school_data['Effective School Leadership Rating'].fillna('missing')
school_data['Average Math Proficiency'] = school_data['Average Math Proficiency'].fillna(school_data['Average Math Proficiency'].mean())
school_data['Average ELA Proficiency'] = school_data['Average ELA Proficiency'].fillna(school_data['Average ELA Proficiency'].mean())


##### Reset index school_data
school_data.reset_index(drop = True, inplace=True)
school_data.shape


There are not enough information available for many schools, to rank them or create a conversion strategy. We have dropped them. If you would like to explore them further, check them here - "school_data_with_missing_values".

In [ ]:
school_data['Community'] = np.where(school_data['Community School?'] == 'Yes', 1, 0)
school_data.shape

In [ ]:
### categorical variable encoding module

def categorical_variable_encoding(df):
    return pd.get_dummies(df, columns=list(df))

for field in ['Rigorous Instruction Rating', 'Collaborative Teachers Rating', 'Supportive Environment Rating', 'Effective School Leadership Rating', 'Strong Family-Community Ties Rating', 'Trust Rating', 'Student Achievement Rating']:
    df = categorical_variable_encoding(school_data[field])
    df.columns = [field + ' ' + s for s in list(df)]
    school_data = school_data.join(df)
school_data.shape

In [ ]:
##### Split school_data into 3 groups #####
group1 = school_data[school_data['proficiency'] == 1].copy()
group2 = school_data[school_data['proficiency'] == 3].copy()
group3 = school_data[school_data['proficiency'] == 4].copy()

group1.reset_index(drop=True, inplace=True)
group2.reset_index(drop=True, inplace=True)
group3.reset_index(drop=True, inplace=True)

##### Convert to array #####

X_group1 = np.array(group1[[    'Early_Childhood',
                                'Elementary',
                                'Middle',
                                'High',
                                'proficiency',
                                'Community',
                                'Economic Need Index',
                                'Percent ELL',
                                'Percent Asian',
                                'Percent Black',
                                'Percent Hispanic',
                                'Percent Black / Hispanic',
                                'Percent White',
                                'Student Attendance Rate',
                                'Percent of Students Chronically Absent',
                                'Rigorous Instruction %',
                                'Collaborative Teachers %',
                                'Supportive Environment %',
                                'Effective School Leadership %',
                                'Strong Family-Community Ties %',
                                'Trust %',
                                'Rigorous Instruction Rating Approaching Target',
                                'Rigorous Instruction Rating Exceeding Target',
                                'Rigorous Instruction Rating Meeting Target',
                                'Rigorous Instruction Rating Not Meeting Target',
                                'Rigorous Instruction Rating missing',
                                'Collaborative Teachers Rating Approaching Target',
                                'Collaborative Teachers Rating Exceeding Target',
                                'Collaborative Teachers Rating Meeting Target',
                                'Collaborative Teachers Rating Not Meeting Target',
                                'Collaborative Teachers Rating missing',
                                'Supportive Environment Rating Approaching Target',
                                'Supportive Environment Rating Exceeding Target',
                                'Supportive Environment Rating Meeting Target',
                                'Supportive Environment Rating Not Meeting Target',
                                'Supportive Environment Rating missing',
                                'Effective School Leadership Rating Approaching Target',
                                'Effective School Leadership Rating Exceeding Target',
                                'Effective School Leadership Rating Meeting Target',
                                'Effective School Leadership Rating Not Meeting Target',
                                'Effective School Leadership Rating missing',
                                'Strong Family-Community Ties Rating Approaching Target',
                                'Strong Family-Community Ties Rating Exceeding Target',
                                'Strong Family-Community Ties Rating Meeting Target',
                                'Strong Family-Community Ties Rating Not Meeting Target',
                                'Strong Family-Community Ties Rating missing',
                                'Trust Rating Approaching Target',
                                'Trust Rating Exceeding Target',
                                'Trust Rating Meeting Target',
                                'Trust Rating Not Meeting Target',
                                'Trust Rating missing',
                                'Student Achievement Rating Approaching Target',
                                'Student Achievement Rating Exceeding Target',
                                'Student Achievement Rating Meeting Target',
                                'Student Achievement Rating Not Meeting Target',
                                'Student Achievement Rating missing',
                                'Average ELA Proficiency',
                                'Average Math Proficiency']])

X_group2 = np.array(group2[[    'Early_Childhood', 
                                'Elementary', 
                                'Middle', 
                                'High',
                                'proficiency',
                                'Community',
                                'Economic Need Index',
                                'Percent ELL',
                                'Percent Asian',
                                'Percent Black',
                                'Percent Hispanic',
                                'Percent Black / Hispanic',
                                'Percent White',
                                'Student Attendance Rate',
                                'Percent of Students Chronically Absent',
                                'Rigorous Instruction %',
                                'Collaborative Teachers %',
                                'Supportive Environment %',
                                'Effective School Leadership %',
                                'Strong Family-Community Ties %',
                                'Trust %',
                                'Rigorous Instruction Rating Approaching Target',
                                'Rigorous Instruction Rating Exceeding Target',
                                'Rigorous Instruction Rating Meeting Target',
                                'Rigorous Instruction Rating Not Meeting Target',
                                'Rigorous Instruction Rating missing',
                                'Collaborative Teachers Rating Approaching Target',
                                'Collaborative Teachers Rating Exceeding Target',
                                'Collaborative Teachers Rating Meeting Target',
                                'Collaborative Teachers Rating Not Meeting Target',
                                'Collaborative Teachers Rating missing',
                                'Supportive Environment Rating Approaching Target',
                                'Supportive Environment Rating Exceeding Target',
                                'Supportive Environment Rating Meeting Target',
                                'Supportive Environment Rating Not Meeting Target',
                                'Supportive Environment Rating missing',
                                'Effective School Leadership Rating Approaching Target',
                                'Effective School Leadership Rating Exceeding Target',
                                'Effective School Leadership Rating Meeting Target',
                                'Effective School Leadership Rating Not Meeting Target',
                                'Effective School Leadership Rating missing',
                                'Strong Family-Community Ties Rating Approaching Target',
                                'Strong Family-Community Ties Rating Exceeding Target',
                                'Strong Family-Community Ties Rating Meeting Target',
                                'Strong Family-Community Ties Rating Not Meeting Target',
                                'Strong Family-Community Ties Rating missing',
                                'Trust Rating Approaching Target',
                                'Trust Rating Exceeding Target',
                                'Trust Rating Meeting Target',
                                'Trust Rating Not Meeting Target',
                                'Trust Rating missing',
                                'Student Achievement Rating Approaching Target',
                                'Student Achievement Rating Exceeding Target',
                                'Student Achievement Rating Meeting Target',
                                'Student Achievement Rating Not Meeting Target',
                                'Student Achievement Rating missing',
                                'Average ELA Proficiency',
                                'Average Math Proficiency']])

X_group3 = np.array(group3[[    'Early_Childhood', 
                                'Elementary', 
                                'Middle', 
                                'High',
                                'proficiency',
                                'Community',
                                'Economic Need Index',
                                'Percent ELL',
                                'Percent Asian',
                                'Percent Black',
                                'Percent Hispanic',
                                'Percent Black / Hispanic',
                                'Percent White',
                                'Student Attendance Rate',
                                'Percent of Students Chronically Absent',
                                'Rigorous Instruction %',
                                'Collaborative Teachers %',
                                'Supportive Environment %',
                                'Effective School Leadership %',
                                'Strong Family-Community Ties %',
                                'Trust %',
                                'Rigorous Instruction Rating Approaching Target',
                                'Rigorous Instruction Rating Exceeding Target',
                                'Rigorous Instruction Rating Meeting Target',
                                'Rigorous Instruction Rating Not Meeting Target',
                                'Rigorous Instruction Rating missing',
                                'Collaborative Teachers Rating Approaching Target',
                                'Collaborative Teachers Rating Exceeding Target',
                                'Collaborative Teachers Rating Meeting Target',
                                'Collaborative Teachers Rating Not Meeting Target',
                                'Collaborative Teachers Rating missing',
                                'Supportive Environment Rating Approaching Target',
                                'Supportive Environment Rating Exceeding Target',
                                'Supportive Environment Rating Meeting Target',
                                'Supportive Environment Rating Not Meeting Target',
                                'Supportive Environment Rating missing',
                                'Effective School Leadership Rating Approaching Target',
                                'Effective School Leadership Rating Exceeding Target',
                                'Effective School Leadership Rating Meeting Target',
                                'Effective School Leadership Rating Not Meeting Target',
                                'Effective School Leadership Rating missing',
                                'Strong Family-Community Ties Rating Approaching Target',
                                'Strong Family-Community Ties Rating Exceeding Target',
                                'Strong Family-Community Ties Rating Meeting Target',
                                'Strong Family-Community Ties Rating Not Meeting Target',
                                'Strong Family-Community Ties Rating missing',
                                'Trust Rating Approaching Target',
                                'Trust Rating Exceeding Target',
                                'Trust Rating Meeting Target',
                                'Trust Rating Not Meeting Target',
                                'Trust Rating missing',
                                'Student Achievement Rating Approaching Target',
                                'Student Achievement Rating Exceeding Target',
                                'Student Achievement Rating Meeting Target',
                                'Student Achievement Rating Not Meeting Target',
                                'Student Achievement Rating missing',
                                'Average ELA Proficiency',
                                'Average Math Proficiency']])

3 nearest neighbour model shown below for Group 2 to Group1 conversion.

Index of kneighbour_matrix represent candidate school from Group1.

Column of kneighbour_matrix represent candidate school from Group2.

In [ ]:
##### Find out 3 nearest(considering above dimensions) school
nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(X_group2)
kneighbour_matrix = nbrs.kneighbors_graph(X_group1).toarray()
kneighbour_matrix.shape

### Select any reference school from Group1 by changing index between 0-142 and execute below two blocks of codes to compare it with 3 nearest neighbours from group2

In [ ]:
group1_school_index = 46
group1.iloc[[group1_school_index]]

In [ ]:
i, = np.where(kneighbour_matrix[group1_school_index] == 1)
group2.iloc[i]

### Value of the i-th postion of below 1D array represent "number of times i-th candidate of Group2 has been selected

In [ ]:
kneighbour_matrix.sum(axis = 0)

In [ ]:
score = list(set(kneighbour_matrix.sum(axis = 0)))
score

### Rank schools in group 2, rank order - top to bottom, highest priority to lowest priority(where lowest marked by 0)

In [ ]:
group2_ranked = pd.DataFrame()
priority = 0
group2_ranked['priority'] = priority
for s in score:
    temp_df = pd.DataFrame()
    r, = np.where(kneighbour_matrix.sum(axis = 0) == s)
    temp_df = temp_df.append(group2.iloc[r])
    temp_df['priority'] = priority
    priority += 1
    group2_ranked = group2_ranked.append(temp_df)
group2_ranked.reset_index(drop = True, inplace = True)
group2_ranked.sort_values(by = 'priority', ascending = False, inplace = True)
group2_ranked.head(5)

Schools grouped by priority and their counts. 0 = Lowest priority.

#### Analysis of results ( Group2 --> Group1 Conversion)

In [ ]:
group2_school_count_by_priority = pd.DataFrame(group2_ranked['priority'].value_counts()).reset_index(drop = False)
group2_school_count_by_priority.columns = ['priority', 'Number of schools']
group2_school_count_by_priority.sort_values(by = 'priority', inplace = True)
group2_school_count_by_priority

In [ ]:
parameters ={
    'Economic Need Index' : 'mean',
    'Percent ELL' : 'mean', 
    'Percent Hispanic' : 'mean', 
    'Percent Black' : 'mean',
    'Percent White' : 'mean',
    'Percent Asian' : 'mean',
    'Percent Black / Hispanic' : 'mean'
    
}

group2_ranked[['priority', 'Economic Need Index', 'Percent ELL', 'Percent Hispanic', 'Percent Black', 'Percent White', 'Percent Asian', 'Percent Black / Hispanic']].groupby('priority').agg(parameters)

# Study : 2nd Part

### District 5 students growth in SHSAT enrollment, registration and appearence over the years

The growth has been smoothen out over the years using Compound Annual Growth Rate (CAGR).

CAGR(%) = [Final Value/Initial Value] ^ (1/Time Period)  - 1

In [ ]:
def calculate_cagr(ser):
    n = len(ser)
    cagr = ((ser.iloc[-1]/ser.iloc[0])**(1/n)) - 1
    return cagr*100

In [ ]:
parameters = {
    'Enrollment on 10/31' : calculate_cagr,
    'Number of students who registered for the SHSAT' : calculate_cagr,
    'Number of students who took the SHSAT' : calculate_cagr
    }

d5_shsat_data[['Enrollment on 10/31', 'Number of students who registered for the SHSAT', 'Number of students who took the SHSAT']] = d5_shsat_data[['Enrollment on 10/31', 'Number of students who registered for the SHSAT', 'Number of students who took the SHSAT']].replace(0, 1)
cagr_d5_shsat = d5_shsat_data[['DBN', 'Grade level', 'Year of SHST', 'Enrollment on 10/31', 'Number of students who registered for the SHSAT', 'Number of students who took the SHSAT']].groupby(['DBN', 'Grade level']).agg(parameters).reset_index(drop = False)


In [ ]:
#cagr_d5_shsat[['Enrollment on 10/31', 'Number of students who registered for the SHSAT', 'Number of students who took the SHSAT']] = cagr_d5_shsat[['Enrollment on 10/31', 'Number of students who registered for the SHSAT', 'Number of students who took the SHSAT']]*100

## Value represents CAGR (%) over the period of 4 years. Growth shown for Grade 8 and 9 separately.

### Grade 8 :

In [ ]:
cagr_d5_shsat_grade8 = cagr_d5_shsat[cagr_d5_shsat['Grade level'] == 8]
cagr_d5_shsat_grade8.reset_index(drop = True, inplace = True)
cagr_d5_shsat_grade8

### Grade 9 :

In [ ]:
cagr_d5_shsat_grade9 = cagr_d5_shsat[cagr_d5_shsat['Grade level'] == 9]
cagr_d5_shsat_grade9.reset_index(drop = True, inplace = True)
cagr_d5_shsat_grade9

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler8 = StandardScaler()
scaler9 = StandardScaler()
cagr_d5_shsat_scaled8 = pd.DataFrame(scaler8.fit_transform(cagr_d5_shsat_grade8[['Enrollment on 10/31', 'Number of students who registered for the SHSAT', 'Number of students who took the SHSAT']]), columns=['Enrollment on 10/31', 'Number of students who registered for the SHSAT', 'Number of students who took the SHSAT'])
cagr_d5_shsat_scaled9 = pd.DataFrame(scaler9.fit_transform(cagr_d5_shsat_grade9[['Enrollment on 10/31', 'Number of students who registered for the SHSAT', 'Number of students who took the SHSAT']]), columns=['Enrollment on 10/31', 'Number of students who registered for the SHSAT', 'Number of students who took the SHSAT'])

Grade 8 - Allmost all schools showing negative growth in enrollment, registration and appearence in the SHSAT.

Grade 9  - Many schools showing negative growth in enrollment, registration and appearence in the exam. Few of them are steady(No +ve or -ve growth).

These discrict 5 schools are estimated to be socio-economically backward, proficient in None(ELA or Math), dominated by Black / Hispanic people, non-white/non-asian and for majority of them English is secondary language. On top of everything, most of them are showing degrading performances participating in SHSAT. They have got 3 community schools.

This group is just a strong example of what we have explored in the first part of the study.

In [ ]:
f,axes=plt.subplots(1,2,figsize=(22,8))
sns.heatmap(cagr_d5_shsat_scaled8, annot = cagr_d5_shsat_grade8[['Enrollment on 10/31', 'Number of students who registered for the SHSAT', 'Number of students who took the SHSAT']], cmap = 'coolwarm_r', ax = axes[0])
axes[0].set_title('\nHeat Map : Grade 8 - CAGR Enrollment, Registration and Appearence \nMore Blue : Stronger +ve growth   More Red : Stronger -ve growth')
sns.heatmap(cagr_d5_shsat_scaled9, annot = cagr_d5_shsat_grade9[['Enrollment on 10/31', 'Number of students who registered for the SHSAT', 'Number of students who took the SHSAT']], cmap = 'coolwarm_r', ax = axes[1])
axes[1].set_title('\nHeat Map : Grade 9 - CAGR Enrollment, Registration and Appearence \nMore Blue : Stronger -ve growth   More Red : Stronger -ve growth')

In [ ]:
#### Enter District Borough Number (dbn) below to check any specific school's figure
dbn = '84M341'
d5_shsat_data[d5_shsat_data['DBN'] == dbn]

In [ ]:
#school_data_d5 = school_data[school_data['Location Code'].isin(d5_shsat_data['DBN'].unique())].copy()
school_data_d5 = school_data[school_data['District'] == 5].copy()
school_data_d5.shape

In [ ]:
f,axes=plt.subplots(4,2,figsize=(24,22))

school_data_d5['proficiency'].hist(ax = axes[0,0])
sns.kdeplot(school_data_d5['Economic Need Index'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[0,1])

sns.kdeplot(school_data_d5['Percent ELL'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[1,0])
sns.kdeplot(school_data_d5['Percent Black'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[1,1])

sns.kdeplot(school_data_d5['Percent White'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[2,0])
sns.kdeplot(school_data_d5['Percent Black / Hispanic'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[2,1])
sns.kdeplot(school_data_d5['Percent Asian'].fillna(0), cut = 0, shade  = True, color = 'r', ax = axes[3,0])

### Realize evolution of proficiency levels, over the years, for various groups

In [ ]:
parameters = {
    'Number Tested' : calculate_cagr,
    'Level 1%' : calculate_cagr,
    'Level 2%' : calculate_cagr,
    'Level 3%' : calculate_cagr,
    'Level 4%' : calculate_cagr,
    'Level 3+4%' : calculate_cagr
    }

cagr_proficiency_shift = school_results[['Category', 'DBN', 'Grade', 'Number Tested', 'Level 1%', 'Level 2%', 'Level 3%', 'Level 4%', 'Level 3+4%']].groupby(['Category','DBN', 'Grade']).agg(parameters).reset_index(drop = False)


#### Change "Grade" and "Category" and run below two code blocks to vizualize the ELA proficiency growth in various groups

Interpretation of results : Ideally is expected, over the years, for each grade, a school will be able to upgrade their ELA proficiency from Level 1% / Level 2% to 
Level 3% /  Level 4%. Therefore, in heatmap leftmost 2 columns are expected to be in RED base and rightmost 2 columns are expected to be in BLUE base if most of the schools are able to do that. Few noise are extected obviously.

Example - 

A. Over the period of 2013-2017, Grade 8 Hispanic student community has been able to upgrade their ELA proficiency. (Left two columns are RED base, Right two columns are BLUE base, noise present)

B. Over the period of 2013-2017, Grade 8 White student community has remain nearly unchanged in their ELA proficiency levels. (All columns are BLUE base, noise present)

In [ ]:
grade = '8' ## Select from 3 to 8
category = 'All Students' ##  Select from 'All Students', 'Asian', 'Black', 'Hispanic', 'White',
              ## 'Econ Disadv', 'Not Econ Disadv', 'Female', 'Male', 'Ever ELL',
              ## 'Never ELL', 'Current ELL', 'Not SWD', 'SWD'
temp_df = cagr_proficiency_shift[(cagr_proficiency_shift['Grade'] == grade) & (cagr_proficiency_shift['Category'] == category)].copy()
temp_df.reset_index(drop = True, inplace = True)

def scale_rows(df):
    return scale(df, axis = 1)

heatmap_input = pd.DataFrame(scale_rows(temp_df[['Level 1%', 'Level 2%', 'Level 3%', 'Level 4%']]), columns = ['Level 1%', 'Level 2%', 'Level 3%', 'Level 4%'])
heatmap_input = heatmap_input.sort_values(by = 'Level 1%')
f,axes=plt.subplots(figsize=(14,4))
sns.heatmap(heatmap_input, cmap = 'coolwarm_r')